In [ ]:
from qdrant_client import QdrantClient, models
client = QdrantClient(url="http://localhost:6333")
client.create_collection(
    collection_name ="{EmbeddingSpace}",
    vectors_config=models.VectorParams(size=384,distance=models.Distance.COSINE)
)

In [ ]:
def retrieveRawData():
    import pymongo
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    local = myclient["local"]
    col =local["RawData"]
    return col.find()
    

def cleanData(data: dict):
    import string
    import re
    content = data["content"]["Content"]
    content = re.sub(r'http\S+', 'URL_PLACEHOLDER', content)   
    content = content.translate(str.maketrans('', '', string.punctuation))
    content = content.encode("ascii", "ignore").decode()
    content = re.sub(r'\s+', ' ', content).strip()  

    data["CleanedContent"] = content
    return data

def chunkData(data: dict):
    content = data["CleanedContent"]
    chunk_size = 500  # This needs to be revised
    chunks = [content[i:i + chunk_size] for i in range(0, len(content), chunk_size)]
    data["Chunks"] = chunks
    return data

def embedData(data: dict):
    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = [model.encode(chunk) for chunk in data["Chunks"]]
    data["Embeddings"] = embeddings
    return data

def loadToQdrant(data: dict):
    import uuid
    from qdrant_client import QdrantClient, models
    client = QdrantClient(url="http://localhost:6333")
    vector_data = []
    for idx, embedding in enumerate(data["Embeddings"]):
        gen_id = str(uuid.uuid4())
        point = models.PointStruct(
            id=gen_id,
            payload={
                'id': f"{data['_id']}_chunk_{idx}",
                'link': data["link"],
                'title': data["content"]["Title"],
                'content_chunk': data["Chunks"][idx]
            },
            vector=embedding
        )
        vector_data.append(point)
    
    client.upsert(collection_name="{EmbeddingSpace}", points=vector_data)
    return data

def mainF():
    docs=retrieveRawData()
    for data in docs:
        data = cleanData(data)
        data = chunkData(data)
        data = embedData(data)
        data = loadToQdrant(data)

mainF()